In [97]:
import sys
import os


# auto-reload library if developing library functionalities
%reload_ext autoreload
%autoreload 2
# Add the old-can-decoder-c56d4fe6 directory to sys.path
sys.path.append(os.path.abspath('old-can-decoder-c56d4fe6'))

# import library
from PandaCANDecoder.decoder import Decoder as OldDecoder
from can_decoder.decoder import Decoder

In [ ]:
decoder = Decoder("./data/SampleRoadAcceleration_v2.csv")

# Generate message objects

byte_filters = {
    "18DA10F1": ["1", "2", "3", "4",'5'],
    "18DAF110": ["1", "2", "3", "4",'5'],
    "18DAF218": ["1", "2", "3", "4",'5'],
    "18DA18F2": ["1", "2", "3", "4",'5'],
}

decoder.generate_msgs(byte_filters=byte_filters)


Generated 910 messages


In [165]:
## Dynamic filtering: if byte 1 is 0 we should discard byte 5
for msg in decoder.msgs:
    if msg.msg_byte_filter is not None:
        if msg.msg_byte_filter['1'] != 0:
            msg.msg_byte_filter.pop('5', None)
            # Remove duplicates

# Remove duplicate messages with same msg_id and identical msg_byte_filter after discarding byte 5
unique_msgs = []
seen = set()
for msg in decoder.msgs:
    key = (msg.msg_id, tuple(sorted(msg.msg_byte_filter.items())) if msg.msg_byte_filter else None)
    if key not in seen:
        unique_msgs.append(msg)
        seen.add(key)
decoder.msgs = unique_msgs

In [169]:
# Generate message time series data

decoder.generate_msg_ts_data(rewrite=True)  

In [170]:
decoder.calculate_signals(
        tokenization_method='conditional_bit_flip',
        signedness_method='msb_classifier',
        alpha1=0.01,
        alpha2=0.5, #0.5,
        gamma1=0.2)

In [ ]:
decoder.get_message('18DAF218',[0x3,0x22,1,201])

Message 18DA10F1 - Filter [B1:3|B2:34|B3:1|B4:201] : Length 8 bytes

In [184]:
decoder.plot_message_from_id('18DAF218',[0x0,19,98,80,58])

In [180]:
decoder.plot_message_from_id('116')

In [178]:
decoder.msgs

[Message 42A: Length 32 bytes,
 Message 18DA10F1 - Filter [B1:3|B2:34|B3:1|B4:201] : Length 8 bytes,
 Message 18DA10F1 - Filter [B1:3|B2:34|B3:1|B4:209] : Length 8 bytes,
 Message 18DA10F1 - Filter [B1:3|B2:34|B3:1|B4:213] : Length 8 bytes,
 Message 18DA10F1 - Filter [B1:3|B2:34|B3:2|B4:178] : Length 8 bytes,
 Message 18DA10F1 - Filter [B1:3|B2:34|B3:2|B4:179] : Length 8 bytes,
 Message 18DA10F1 - Filter [B1:3|B2:34|B3:2|B4:180] : Length 8 bytes,
 Message 18DA10F1 - Filter [B1:3|B2:34|B3:2|B4:181] : Length 8 bytes,
 Message 18DA10F1 - Filter [B1:3|B2:34|B3:3|B4:225] : Length 8 bytes,
 Message 18DA10F1 - Filter [B1:3|B2:34|B3:3|B4:240] : Length 8 bytes,
 Message 18DA10F1 - Filter [B1:3|B2:34|B3:1|B4:29] : Length 8 bytes,
 Message 18DA10F1 - Filter [B1:3|B2:34|B3:1|B4:32] : Length 8 bytes,
 Message 18DA10F1 - Filter [B1:3|B2:34|B3:1|B4:35] : Length 8 bytes,
 Message 18DA10F1 - Filter [B1:3|B2:34|B3:1|B4:39] : Length 8 bytes,
 Message 18DA10F1 - Filter [B1:3|B2:34|B3:18|B4:201] : Length 8

In [181]:
0x50, 0x3A

(80, 58)

In [69]:
all_data = decoder.all_data